This is the program I made for sending bulk billing emails. It has sent hundreds of thousands of emails in bulk. It has greatly improved operational efficiency.

In [ ]:


import os
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import (
    Mail, Attachment, FileContent, FileName,
    FileType, Disposition, ContentId)
import email, smtplib, ssl
import argparse
from os.path import join, getsize
import xlrd
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import logging
import time
import base64
import pandas as pd

logging.basicConfig(filename='log/send_invoice_to_merchants.log', format='%(asctime)s | %(levelname)s | %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler())

parser = argparse.ArgumentParser(description='Send email to merchants')
parser.add_argument('--mode', choices=['test', 'prod'], help='Two modes are available: test | prod', default='prod')

args = parser.parse_args()
mode = args.mode

logging.info('Sending email in mode: {}'.format(mode))

API_KEY = input("Please enter your SendGrid API key: ")

if mode == 'prod':
    # Prod email
    email = "financeteam.usa@hungrypanda.co"
else:
    # Test email
    email = "financeteam.usa@hungrypanda.co"


def attach_file(message, file_path, filename):
    with open(file_path, 'rb') as f:
        data = f.read()
        f.close()
    encoded = base64.b64encode(data).decode()
    attachment = Attachment()
    attachment.file_content = FileContent(encoded)
    attachment.file_type = FileType('application/{}'.format(filename.split(".")[1]))
    attachment.file_name = FileName(filename)
    attachment.disposition = Disposition('attachment')
    attachment.content_id = ContentId(filename)
    message.add_attachment(attachment)


def main():
    sg = SendGridAPIClient(API_KEY)
    # change information
    info_path = input('Excel Path: ')
    pdf_path = input('PDF Path: ')
    info = pd.read_excel(info_path, usecols=['ID', 'Email', 'Statement Period'])

    # info = info.fillna(0)
    for index, shop_info in info.iterrows():
        mail_msg = '''
        <p>Hi,</p>

<p>Please find attached for your California Prop22 Driver Earning Adjustment Statement ({}). </p>

<p>Should you have any questions, please contact our local delivery management team. Thank you.</p>

<p>Best regards,</p>
<p>HungryPanda Finance Team</p>
        '''.format(shop_info['Statement Period'])
        subject = 'HungryPanda California Prop22 Driver Earning Adjustment Statement（{}）'.format(
                                                                          shop_info['Statement Period'].replace('/', ''))
        try:
            receiver = str(shop_info['Email'])
        except:
            print('ignore: ' + index)

        if mode != 'prod':
            # test email
            receiver = "financejp@hungrypanda.co"

        try:
            message = Mail(
                from_email=email,
                to_emails=receiver,
                subject=subject,
                html_content=mail_msg
            )
            file = '{}-{}.pdf'.format(str(shop_info['ID']), shop_info['Statement Period'].replace('/', ''))
            full_path = '{}/{}'.format(pdf_path, file)
            attach_file(message, full_path, file)
            response = sg.send(message)
            logging.info(response)
        except Exception as e:
            logging.info("Send email failed: " + receiver)


if __name__ == '__main__':
    main()